In [11]:
# Import necessary libraries

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout
from tensorflow.keras.utils import text_dataset_from_directory
import re

In [7]:
# Setup some initial parameters
batch_size = 32
validation_ratio = 0.2
seed = 42
max_features = 10000
sequence_length = 250

In [8]:
# Load datasets
raw_train_ds = text_dataset_from_directory(
    "temp/train",
    batch_size=batch_size,
    validation_split=validation_ratio,
    subset='training',
    seed=seed
)
raw_val_ds = text_dataset_from_directory(
    'temp/train',
    batch_size=batch_size,
    validation_split=validation_ratio,
    subset='validation',
    seed=seed
)
raw_test_ds = text_dataset_from_directory(
    'temp/test',
    batch_size=batch_size
)

Found 3124 files belonging to 131 classes.
Using 2500 files for training.
Found 3124 files belonging to 131 classes.
Using 624 files for validation.
Found 347 files belonging to 80 classes.


In [9]:
# Define a custom text standardization function to preprocess text data
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'), '')

In [12]:
# Create a TextVectorization layer to vectorize text
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length
)
vectorize_layer.adapt(raw_train_ds.map(lambda text, label: text))

In [13]:
# Define the model architecture
model = Sequential([
    vectorize_layer,
    Embedding(max_features + 1, 16),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(1)  # Output layer for regression; no activation function
])

In [14]:
# Compile the model
model.compile(
    loss="mean_squared_error",  # Using MSE for regression
    optimizer="adam",
    metrics=["mean_absolute_error"]  # Using MAE as a metric for regression
)

In [15]:
# Train the model
epochs = 100
model.fit(
    raw_train_ds,
    validation_data=raw_val_ds,
    epochs=epochs
)

Epoch 1/100
79/79 [==============================] - 1s 5ms/step - loss: 3716.8823 - mean_absolute_error: 54.1890 - val_loss: 2956.2852 - val_mean_absolute_error: 47.0073
Epoch 2/100
79/79 [==============================] - 0s 5ms/step - loss: 1248.7375 - mean_absolute_error: 27.3123 - val_loss: 808.6818 - val_mean_absolute_error: 21.4769
Epoch 3/100
79/79 [==============================] - 0s 5ms/step - loss: 779.9658 - mean_absolute_error: 21.4694 - val_loss: 802.2999 - val_mean_absolute_error: 21.4383
Epoch 4/100
79/79 [==============================] - 0s 5ms/step - loss: 750.5126 - mean_absolute_error: 20.9782 - val_loss: 808.1620 - val_mean_absolute_error: 21.2962
Epoch 5/100
79/79 [==============================] - 0s 5ms/step - loss: 738.8555 - mean_absolute_error: 20.8027 - val_loss: 803.3452 - val_mean_absolute_error: 21.3360
Epoch 6/100
79/79 [==============================] - 0s 6ms/step - loss: 712.8732 - mean_absolute_error: 20.4521 - val_loss: 807.3243 - val_mean_absolut

In [16]:
# Evaluate the model
model.evaluate(raw_test_ds)

11/11 [==============================] - 0s 3ms/step - loss: 828.1508 - mean_absolute_error: 26.3339


[828.1507568359375, 26.333877563476562]